In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from urllib.request import urlopen
import urllib

from selenium import webdriver
import openpyxl
import time
import math
from collections import Counter


from tqdm.notebook import tqdm


In [2]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2 if len(d.replace("=",":").split(":")) != 1]
    params_ = {}
    #print(para_3)
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [3]:
ua = UserAgent()

# Main Page Crawling

In [4]:
def Main_page_head(q_1,p_1):
    url_2 = "http://www.lotteimall.com/search/searchMain.lotte?headerQuery={}&page={}&requery=&searchType=S&cate_depth=1&lvl1_cate_no=&lvl2_cate_no=&lvl3_cate_no=&lvl4_cate_no=&lvl5_cate_no=&color_cd=&brand_no=&brand_sort=RANK%2FDESC&min_price=&max_price=&check_tvShop=&gdas_cnt_yn=&inst_cpn_yn=&dlv_free_yn=&smtp_yn=&stpup_yn=&stap_yn=&stap_cd=&size_cd=&gender_cd=&stpup_entr_contr_no=&dpml_no=&colldisplay=60&lst_sort_cd=GDAS_CNT%2FDESC&cate_depth=1&lvl1_cate_no=&lvl2_cate_no="

    url_2_2 = url_2.format(q_1,p_1)
    html = requests.get(url_2_2).text
    soup = BeautifulSoup(html,"html.parser")
    item_list,name_head,reveiw_count_1 = [],[],[]
    for tag in soup.select(".wrap_unitlist ul li"):
        item_list.append(tag["goodsno"])
        name_head.append(tag.find("p",class_="title").text.strip())
        reveiw_count_1.append(int(tag.find("p",class_="info2").find("span",class_="comment").text.replace("상품평","").strip().replace(",","")))
    return pd.DataFrame([name_head,item_list,reveiw_count_1]).T.rename(columns={0:"Name",1:"item_num",2:"Review_count"})

In [5]:
main_df = Main_page_head("헤어드라이기",1)
main_df.head()

,Name,item_num,Review_count
0,[공식] 슈퍼소닉 드라이어 HD03 아이언핑크,12610778,1166
1,팸퍼스 베이비드라이 밴드 2/3/4/5/6단계 1박스 기저귀,1409491229,693
2,팸퍼스 베이비드라이 팬티 3/4/5/6단계 4팩 기저귀,1472398945,537
3,JMW M5001A PLUS 전문가용 항공모터 헤어 드라이기,1630255408,529
4,JMW M5001A PLUS 전문가용 항공모터 헤어 드라이기+사은품,1630257815,509


# Review page crawling

In [6]:
def Review_page_ex(target_name,item_n,p_2):
    main_2 = 'http://www.lotteimall.com/goods/searchMultiGdasInfo.lotte'
    url_main = 'http://www.lotteimall.com/goods/searchPrGdasInfoList.lotte'
    params = {'goods_no': item_n,
     'gdas_no': '',
     'hdr_tp_cd': '10',
     'rowsPerPage': '200',
     'pageIdx': p_2,
     'opt_desc': '',
     'size_desc': '',
     'gdas_stfd_val': '',
     'gdasAvgScore': '99',
     'totalCnt': '236',
     'healthfood_yn': 'N',
     'healthfood_disp': 'N',
     'tap_no': '0',
     'multi': 'all',
     'isEcoupon': 'N',
     'ec_goods_artc_cd': '108',
     'gdasSelectImgIndex': '',
     'list_no': '',
     'preb': ''}

    html = requests.get(url_main,params=params).text
    soup = BeautifulSoup(html,"html.parser")

    data_all = []
    for tag in soup.select(".list_comment li"):
        id_,date_ = tag.find("div",class_="info_list").find("p").text.split("|")
        item_ = target_name
        score_ = int(tag.find("div",class_="info_list").find("div").find('div')["class"][2].replace("s",""))/10
        contents_ = tag.find("div",class_="cont").find("span").text
        liked_ = ""
        channel_ = "LOTTE"
        all_ = [date_,id_,item_,score_,contents_,liked_,channel_]
        data_all.append(all_)
    df_ = pd.DataFrame(data_all).rename(columns={0:"date",1:"id",2:"item",3:"score",4:"contents",5:"liked",6:"channel"})
    return df_


In [7]:
Ta_ = 3
item_n = main_df["item_num"][Ta_]
target_name = main_df["Name"][Ta_]
max_page = main_df["Review_count"][Ta_]

Review_page_ex(target_name,item_n,2).head()

,date,id,item,score,contents,liked,channel
0,2017/01/21,kwa*****@hanmail.net,JMW M5001A PLUS 전문가용 항공모터 헤어 드라이기,5.0,좋은제품감사해요~사업번창하세요~,,LOTTE
1,2017/01/21,nih****,JMW M5001A PLUS 전문가용 항공모터 헤어 드라이기,5.0,만족합니다 추천합니다,,LOTTE
2,2017/01/20,sib****,JMW M5001A PLUS 전문가용 항공모터 헤어 드라이기,5.0,머리숱 많은데 머리말리는 시간이 줄어서 만족해요\r크기만 좀 작았어도,,LOTTE
3,2017/01/20,ans*****@hanmail.net,JMW M5001A PLUS 전문가용 항공모터 헤어 드라이기,5.0,만족합니다~수고하세요,,LOTTE
4,2017/01/20,unn*****,JMW M5001A PLUS 전문가용 항공모터 헤어 드라이기,5.0,완전 짱입니다\r아침마다 머리카락 말리는데 시간 많이 걸려서 힘들었는데 시간도 단축...,,LOTTE


In [8]:
def Crawling_Lotte_all(main_df,Ta_):
    item_n = main_df["item_num"][Ta_]
    target_name = main_df["Name"][Ta_]
    max_page = main_df["Review_count"][Ta_]

    MaxPage = max_page//200
    tq = tqdm(range(MaxPage+1))

    review_all = pd.DataFrame()
    for mp in tq:
        review_candi = Review_page_ex(target_name,item_n,mp+1)
        review_all = pd.concat([review_all,review_candi]).reset_index(drop=True)
    return review_all,target_name

In [9]:
def Lotte_Save_local(path_,head_df):
    for i_ in range(len(head_df)):
        print("In {} Pro From {}".format(i_+1,len(head_df)+1))
        print()
        print(head_df["Name"][i_])
        out_,NA_ = Crawling_Lotte_all(head_df,i_)
        NA_ = NA_.replace("/","")
        out_.to_excel(path_.format(NA_))

In [11]:
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping_2\LOTTE\{}.xlsx"
Lotte_Save_local(path_,main_df)

In 1 Pro From 61

[공식] 슈퍼소닉 드라이어 HD03 아이언핑크


  0%|          | 0/6 [00:00<?, ?it/s]

In 2 Pro From 61

팸퍼스 베이비드라이 밴드 2/3/4/5/6단계 1박스 기저귀


  0%|          | 0/4 [00:00<?, ?it/s]

In 3 Pro From 61

팸퍼스 베이비드라이 팬티 3/4/5/6단계 4팩 기저귀


...

  0%|          | 0/1 [00:00<?, ?it/s]

In 59 Pro From 61

푸마 여성 드라이셀 멀티 이지웨어 3세트(상의 2종 + 하의 2종+원피스1종+레깅스1종)


  0%|          | 0/1 [00:00<?, ?it/s]

In 60 Pro From 61

피트엑스 드라이쿨 남녀 베이직 덧신 3매세트/페이크삭스/기능성원사/빅사이즈


  0%|          | 0/1 [00:00<?, ?it/s]